In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Fuzzy Matching").getOrCreate()

In [0]:
%pip install jellyfish

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# dbutils.library.restartPython()

In [0]:
spark.version

'4.0.0'

In [0]:
data = [
    ("Karim Al-Masri ", "Syrian", "1982-07-17", "Black Crescent", "High", "Aleppo, Syria", "At Large"),
    (" Aisha Rahmani", "Pakistani", "1990-03-22", "Crimson Vow", "Medium", "Karachi, Pakistan", "Captured"),
    ("Omar Haddad  ", "Iraqi", "1975-11-05", "Desert Flame", "High", "Mosul, Iraq", "Deceased"),
    ("Fatima Noor", "Afghan", "1988-06-30", "Silent Dagger", "Low", "Kabul, Afghanistan", "At Large"),
    ("Yusuf Khan ", "Indian", "19800113", "Red Scorpion", "High", "Srinagar, India", "Captured"),
    ("Leila Mansour", "Lebanese", "1992/09/18", "Shadow Syndicate", "Medium", "Beirut, Lebanon", "At Large"),
    (" Tariq Al-Fulan", "Jordanian", "6/4/1985", "Phoenix Order", "High", "Amman, Jordan", "Deceased"),
    ("Nadia Qureshi", "Bangladeshi", "1993-12-03", "Crimson Vow", "Low", "Dhaka, Bangladesh", "Captured"),
    ("Bilal Sheikh  ", "Egyptian", "09/09/1987", "Black Crescent", "Medium", "Cairo, Egypt", "At Large"),
    ("  Samira Darwish", "Moroccan", "27-05-1986", "Desert Flame", "High", "Rabat, Morocco", "At Large")
]

columnss = ("Name", "Nationality", "DOB_bl", "Organization", "Threat Level", "Last Known Location", "Status")

df_blacklisted = spark.createDataFrame(data, columnss)

In [0]:
df_blacklisted.show()

+----------------+-----------+----------+----------------+------------+-------------------+--------+
|            Name|Nationality|    DOB_bl|    Organization|Threat Level|Last Known Location|  Status|
+----------------+-----------+----------+----------------+------------+-------------------+--------+
| Karim Al-Masri |     Syrian|1982-07-17|  Black Crescent|        High|      Aleppo, Syria|At Large|
|   Aisha Rahmani|  Pakistani|1990-03-22|     Crimson Vow|      Medium|  Karachi, Pakistan|Captured|
|   Omar Haddad  |      Iraqi|1975-11-05|    Desert Flame|        High|        Mosul, Iraq|Deceased|
|     Fatima Noor|     Afghan|1988-06-30|   Silent Dagger|         Low| Kabul, Afghanistan|At Large|
|     Yusuf Khan |     Indian|  19800113|    Red Scorpion|        High|    Srinagar, India|Captured|
|   Leila Mansour|   Lebanese|1992/09/18|Shadow Syndicate|      Medium|    Beirut, Lebanon|At Large|
|  Tariq Al-Fulan|  Jordanian|  6/4/1985|   Phoenix Order|        High|      Amman, Jordan|

In [0]:
policy_holder_data = [
    ("Rahmani Aisha  ", "22/03/1990", "Pakistani", "Inactive"),
    ("Haddad Omar", "05-11-1975", "Iraqi", "Active"),
    ("Yusuf K.", "1980-01-12", "Indian", "Inactive"),
    ("Mansour Leila ", "1992-09-18", "Lebanese", "Active"),
    ("Bhagyashree Khade", "1983-07-22", "Indian", "Active"),
    ("Bilal S.", "1978/08/09", "Egyptian", "Active"),
    ("Anil Rawat", "1982-01-16", "Indian", "Inactive"),
    ("Darwish Samira", "27/05/1986", "Moroccan", "Inactive"),
    ("Kareem Almasri ", "1982-07-15", "Syrian", "Active"),
    ("Ayesha Rahman", "1990/03/22", "Pakistani", "Active"),
    ("Noor Fatima", "1988-06-30", "Afghan", "Active"),
    (" Khan Yusuf", "1980-01-12", "Indian", "Active"),
    ("Saurabh Kumar Gupta", "1980-01-12", "Indian", "Active")
]

columns = ["Full Name", "DOB_ph", "Nationality", "Status"]

df_policyholder = spark.createDataFrame(policy_holder_data, columns)

In [0]:
df_policyholder.show()

+-------------------+----------+-----------+--------+
|          Full Name|    DOB_ph|Nationality|  Status|
+-------------------+----------+-----------+--------+
|    Rahmani Aisha  |22/03/1990|  Pakistani|Inactive|
|        Haddad Omar|05-11-1975|      Iraqi|  Active|
|           Yusuf K.|1980-01-12|     Indian|Inactive|
|     Mansour Leila |1992-09-18|   Lebanese|  Active|
|  Bhagyashree Khade|1983-07-22|     Indian|  Active|
|           Bilal S.|1978/08/09|   Egyptian|  Active|
|         Anil Rawat|1982-01-16|     Indian|Inactive|
|     Darwish Samira|27/05/1986|   Moroccan|Inactive|
|    Kareem Almasri |1982-07-15|     Syrian|  Active|
|      Ayesha Rahman|1990/03/22|  Pakistani|  Active|
|        Noor Fatima|1988-06-30|     Afghan|  Active|
|         Khan Yusuf|1980-01-12|     Indian|  Active|
|Saurabh Kumar Gupta|1980-01-12|     Indian|  Active|
+-------------------+----------+-----------+--------+



In [0]:
print(df_blacklisted.count())
print(df_policyholder.count())

10
13


In [0]:
display(df_blacklisted)
display(df_policyholder)

Name,Nationality,DOB_bl,Organization,Threat Level,Last Known Location,Status
Karim Al-Masri,Syrian,1982-07-17,Black Crescent,High,"Aleppo, Syria",At Large
Aisha Rahmani,Pakistani,1990-03-22,Crimson Vow,Medium,"Karachi, Pakistan",Captured
Omar Haddad,Iraqi,1975-11-05,Desert Flame,High,"Mosul, Iraq",Deceased
Fatima Noor,Afghan,1988-06-30,Silent Dagger,Low,"Kabul, Afghanistan",At Large
Yusuf Khan,Indian,19800113,Red Scorpion,High,"Srinagar, India",Captured
Leila Mansour,Lebanese,1992/09/18,Shadow Syndicate,Medium,"Beirut, Lebanon",At Large
Tariq Al-Fulan,Jordanian,6/4/1985,Phoenix Order,High,"Amman, Jordan",Deceased
Nadia Qureshi,Bangladeshi,1993-12-03,Crimson Vow,Low,"Dhaka, Bangladesh",Captured
Bilal Sheikh,Egyptian,09/09/1987,Black Crescent,Medium,"Cairo, Egypt",At Large
Samira Darwish,Moroccan,27-05-1986,Desert Flame,High,"Rabat, Morocco",At Large


Full Name,DOB_ph,Nationality,Status
Rahmani Aisha,22/03/1990,Pakistani,Inactive
Haddad Omar,05-11-1975,Iraqi,Active
Yusuf K.,1980-01-12,Indian,Inactive
Mansour Leila,1992-09-18,Lebanese,Active
Bhagyashree Khade,1983-07-22,Indian,Active
Bilal S.,1978/08/09,Egyptian,Active
Anil Rawat,1982-01-16,Indian,Inactive
Darwish Samira,27/05/1986,Moroccan,Inactive
Kareem Almasri,1982-07-15,Syrian,Active
Ayesha Rahman,1990/03/22,Pakistani,Active


In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, split, trim

df_blacklisted_standardized = df_blacklisted.withColumn("standardized_names", lower(regexp_replace(trim(col("Name")), "[^a-zA-Z\\s]", "")))

df_policyholder_standardized = df_policyholder.withColumn("standardized_names", lower(regexp_replace(trim(col("Full Name")), "[^a-zA-Z\\s]", "")))

df_blacklisted_standardized.show()
df_policyholder_standardized.show()

+----------------+-----------+----------+----------------+------------+-------------------+--------+------------------+
|            Name|Nationality|    DOB_bl|    Organization|Threat Level|Last Known Location|  Status|standardized_names|
+----------------+-----------+----------+----------------+------------+-------------------+--------+------------------+
| Karim Al-Masri |     Syrian|1982-07-17|  Black Crescent|        High|      Aleppo, Syria|At Large|     karim almasri|
|   Aisha Rahmani|  Pakistani|1990-03-22|     Crimson Vow|      Medium|  Karachi, Pakistan|Captured|     aisha rahmani|
|   Omar Haddad  |      Iraqi|1975-11-05|    Desert Flame|        High|        Mosul, Iraq|Deceased|       omar haddad|
|     Fatima Noor|     Afghan|1988-06-30|   Silent Dagger|         Low| Kabul, Afghanistan|At Large|       fatima noor|
|     Yusuf Khan |     Indian|  19800113|    Red Scorpion|        High|    Srinagar, India|Captured|        yusuf khan|
|   Leila Mansour|   Lebanese|1992/09/18

In [0]:
df_blacklisted_tokenized = df_blacklisted_standardized.withColumn("tokenized_names_blacklisted", split(col("standardized_names"), " "))

df_policyholder_tokenized = df_policyholder_standardized.withColumn("tokenized_names_policyholder", split(col("standardized_names"), " "))

display(df_blacklisted_tokenized)
display(df_policyholder_tokenized)

Name,Nationality,DOB_bl,Organization,Threat Level,Last Known Location,Status,standardized_names,tokenized_names_blacklisted
Karim Al-Masri,Syrian,1982-07-17,Black Crescent,High,"Aleppo, Syria",At Large,karim almasri,"List(karim, almasri)"
Aisha Rahmani,Pakistani,1990-03-22,Crimson Vow,Medium,"Karachi, Pakistan",Captured,aisha rahmani,"List(aisha, rahmani)"
Omar Haddad,Iraqi,1975-11-05,Desert Flame,High,"Mosul, Iraq",Deceased,omar haddad,"List(omar, haddad)"
Fatima Noor,Afghan,1988-06-30,Silent Dagger,Low,"Kabul, Afghanistan",At Large,fatima noor,"List(fatima, noor)"
Yusuf Khan,Indian,19800113,Red Scorpion,High,"Srinagar, India",Captured,yusuf khan,"List(yusuf, khan)"
Leila Mansour,Lebanese,1992/09/18,Shadow Syndicate,Medium,"Beirut, Lebanon",At Large,leila mansour,"List(leila, mansour)"
Tariq Al-Fulan,Jordanian,6/4/1985,Phoenix Order,High,"Amman, Jordan",Deceased,tariq alfulan,"List(tariq, alfulan)"
Nadia Qureshi,Bangladeshi,1993-12-03,Crimson Vow,Low,"Dhaka, Bangladesh",Captured,nadia qureshi,"List(nadia, qureshi)"
Bilal Sheikh,Egyptian,09/09/1987,Black Crescent,Medium,"Cairo, Egypt",At Large,bilal sheikh,"List(bilal, sheikh)"
Samira Darwish,Moroccan,27-05-1986,Desert Flame,High,"Rabat, Morocco",At Large,samira darwish,"List(samira, darwish)"


Full Name,DOB_ph,Nationality,Status,standardized_names,tokenized_names_policyholder
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)"
Haddad Omar,05-11-1975,Iraqi,Active,haddad omar,"List(haddad, omar)"
Yusuf K.,1980-01-12,Indian,Inactive,yusuf k,"List(yusuf, k)"
Mansour Leila,1992-09-18,Lebanese,Active,mansour leila,"List(mansour, leila)"
Bhagyashree Khade,1983-07-22,Indian,Active,bhagyashree khade,"List(bhagyashree, khade)"
Bilal S.,1978/08/09,Egyptian,Active,bilal s,"List(bilal, s)"
Anil Rawat,1982-01-16,Indian,Inactive,anil rawat,"List(anil, rawat)"
Darwish Samira,27/05/1986,Moroccan,Inactive,darwish samira,"List(darwish, samira)"
Kareem Almasri,1982-07-15,Syrian,Active,kareem almasri,"List(kareem, almasri)"
Ayesha Rahman,1990/03/22,Pakistani,Active,ayesha rahman,"List(ayesha, rahman)"


In [0]:
from pyspark.sql.functions import broadcast
joined_df = df_policyholder_tokenized.crossJoin(broadcast(df_blacklisted_tokenized))

In [0]:
display(joined_df)

Full Name,DOB_ph,Nationality,Status,standardized_names,tokenized_names_policyholder,Name,Nationality,DOB_bl,Organization,Threat Level,Last Known Location,Status,standardized_names,tokenized_names_blacklisted
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Karim Al-Masri,Syrian,1982-07-17,Black Crescent,High,"Aleppo, Syria",At Large,karim almasri,"List(karim, almasri)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Aisha Rahmani,Pakistani,1990-03-22,Crimson Vow,Medium,"Karachi, Pakistan",Captured,aisha rahmani,"List(aisha, rahmani)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Omar Haddad,Iraqi,1975-11-05,Desert Flame,High,"Mosul, Iraq",Deceased,omar haddad,"List(omar, haddad)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Fatima Noor,Afghan,1988-06-30,Silent Dagger,Low,"Kabul, Afghanistan",At Large,fatima noor,"List(fatima, noor)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Yusuf Khan,Indian,19800113,Red Scorpion,High,"Srinagar, India",Captured,yusuf khan,"List(yusuf, khan)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Leila Mansour,Lebanese,1992/09/18,Shadow Syndicate,Medium,"Beirut, Lebanon",At Large,leila mansour,"List(leila, mansour)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Tariq Al-Fulan,Jordanian,6/4/1985,Phoenix Order,High,"Amman, Jordan",Deceased,tariq alfulan,"List(tariq, alfulan)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Nadia Qureshi,Bangladeshi,1993-12-03,Crimson Vow,Low,"Dhaka, Bangladesh",Captured,nadia qureshi,"List(nadia, qureshi)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Bilal Sheikh,Egyptian,09/09/1987,Black Crescent,Medium,"Cairo, Egypt",At Large,bilal sheikh,"List(bilal, sheikh)"
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Samira Darwish,Moroccan,27-05-1986,Desert Flame,High,"Rabat, Morocco",At Large,samira darwish,"List(samira, darwish)"


In [0]:
from pyspark.sql.functions import udf, max
from pyspark.sql.types import DoubleType, ArrayType, StringType
import jellyfish


@udf(DoubleType())
def get_match_score(ph_tokens, bl_tokens):
    if (ph_tokens is None) or (bl_tokens is None) or (not ph_tokens) or (not bl_tokens):
        return 0.0
    
    best_match_score = []
    for ph_token in ph_tokens:
        max_score = 0.0
        for bl_token in bl_tokens:
            try:
                score = jellyfish.jaro_winkler(ph_token, bl_token) 
            except AttributeError:
                # Fallback check in case your cluster has an older version
                score = jellyfish.jaro_winkler_similarity(ph_token, bl_token)
            if score > max_score:
                max_score = score
        best_match_score.append(max_score)
    return sum(best_match_score) / len(best_match_score)

In [0]:
joined_df = joined_df.withColumn("match_score", get_match_score(col("tokenized_names_policyholder"),
        col("tokenized_names_blacklisted")))

In [0]:
display(joined_df.limit(50))

Full Name,DOB_ph,Nationality,Status,standardized_names,tokenized_names_policyholder,Name,Nationality,DOB_bl,Organization,Threat Level,Last Known Location,Status,standardized_names,tokenized_names_blacklisted,match_score
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Karim Al-Masri,Syrian,1982-07-17,Black Crescent,High,"Aleppo, Syria",At Large,karim almasri,"List(karim, almasri)",0.6396825396825396
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Aisha Rahmani,Pakistani,1990-03-22,Crimson Vow,Medium,"Karachi, Pakistan",Captured,aisha rahmani,"List(aisha, rahmani)",1.0
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Omar Haddad,Iraqi,1975-11-05,Desert Flame,High,"Mosul, Iraq",Deceased,omar haddad,"List(omar, haddad)",0.5547619047619048
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Fatima Noor,Afghan,1988-06-30,Silent Dagger,Low,"Kabul, Afghanistan",At Large,fatima noor,"List(fatima, noor)",0.6714285714285715
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Yusuf Khan,Indian,19800113,Red Scorpion,High,"Srinagar, India",Captured,yusuf khan,"List(yusuf, khan)",0.5408730158730158
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Leila Mansour,Lebanese,1992/09/18,Shadow Syndicate,Medium,"Beirut, Lebanon",At Large,leila mansour,"List(leila, mansour)",0.5238095238095238
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Tariq Al-Fulan,Jordanian,6/4/1985,Phoenix Order,High,"Amman, Jordan",Deceased,tariq alfulan,"List(tariq, alfulan)",0.5904761904761905
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Nadia Qureshi,Bangladeshi,1993-12-03,Crimson Vow,Low,"Dhaka, Bangladesh",Captured,nadia qureshi,"List(nadia, qureshi)",0.5809523809523809
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Bilal Sheikh,Egyptian,09/09/1987,Black Crescent,Medium,"Cairo, Egypt",At Large,bilal sheikh,"List(bilal, sheikh)",0.5238095238095238
Rahmani Aisha,22/03/1990,Pakistani,Inactive,rahmani aisha,"List(rahmani, aisha)",Samira Darwish,Moroccan,27-05-1986,Desert Flame,High,"Rabat, Morocco",At Large,samira darwish,"List(samira, darwish)",0.6908730158730159


In [0]:
from pyspark.sql.functions import col, max, rank
from pyspark.sql.window import Window

window_spec = Window.partitionBy(col("Full Name")).orderBy(col("match_score").desc())

ranked_df = joined_df.withColumn("rnk", rank().over(window_spec))

In [0]:
final_results = ranked_df.filter(col("rnk") == 1)

In [0]:
name_scored_df = final_results.select(col("Full Name").alias("ph_name"), "DOB_ph", col("Name").alias("bl_name"), "DOB_bl", (col("match_score") * 100).alias("name_match_score_percentage"))

In [0]:
spark.conf.set("spark.sql.ansi.enabled", "false")

In [0]:
from pyspark.sql.functions import coalesce, to_date
# from pyspark.sql.functions import coalesce, try_to_timestamp


DATE_FORMATS = [
    "yyyy-MM-dd",    # Standard SQL/ISO format
    "MM/dd/yyyy",    # US format
    "dd-MM-yyyy",    # European format
    "M/d/yyyy",      # US short format
    "yyyy/MM/dd",
    "dd/MM/yyyy",
    "yyyyMMdd"       # no separator format
]

def standardize_dob_column(df, column_name):
    """
    Attempts to convert a string column to DateType using multiple formats.
    """
    # Create a list of to_date expressions, one for each format
    date_expressions = [to_date(col(column_name), f) for f in DATE_FORMATS]
    # date_expressions = [try_to_timestamp(col(column_name), f) for f in DATE_FORMATS]

    
    # Use coalesce to pick the first non-null date result
    return df.withColumn(
        column_name + "_standardized", 
        coalesce(*date_expressions)
    )

In [0]:
# from pyspark.sql.functions import coalesce, to_date

dob_standardized = standardize_dob_column(name_scored_df, "DOB_ph")
dob_standardized = standardize_dob_column(dob_standardized, "DOB_bl")
display(dob_standardized)

ph_name,DOB_ph,bl_name,DOB_bl,name_match_score_percentage,DOB_ph_standardized,DOB_bl_standardized
Khan Yusuf,1980-01-12,Yusuf Khan,19800113,100.0,1980-01-12,1980-01-13
Anil Rawat,1982-01-16,Aisha Rahmani,1990-03-22,65.47619047619047,1982-01-16,1990-03-22
Ayesha Rahman,1990/03/22,Aisha Rahmani,1990-03-22,90.57142857142857,1990-03-22,1990-03-22
Bhagyashree Khade,1983-07-22,Yusuf Khan,19800113,70.4469696969697,1983-07-22,1980-01-13
Bilal S.,1978/08/09,Bilal Sheikh,09/09/1987,87.5,1978-08-09,1987-09-09
Darwish Samira,27/05/1986,Samira Darwish,27-05-1986,100.0,1986-05-27,1986-05-27
Haddad Omar,05-11-1975,Omar Haddad,1975-11-05,100.0,1975-11-05,1975-11-05
Kareem Almasri,1982-07-15,Karim Al-Masri,1982-07-17,93.77777777777779,1982-07-15,1982-07-17
Mansour Leila,1992-09-18,Leila Mansour,1992/09/18,100.0,1992-09-18,1992-09-18
Noor Fatima,1988-06-30,Fatima Noor,1988-06-30,100.0,1988-06-30,1988-06-30


In [0]:
from pyspark.sql.functions import when, year

df_with_dob_score = dob_standardized.withColumn(
    "dob_match_score",
    when(col("DOB_ph_standardized") == col("DOB_bl_standardized"), 100)  # Exact Match (100%)
    
    # Check for Year match only (common typo)
    .when((year(col("DOB_ph_standardized")) == year(col("DOB_bl_standardized"))), 50)  # Close Match (50%)
    
    .otherwise(0) # No Match (0%)
)

# # 2. Convert to percentage
# df_with_dob_score = df_with_dob_score.withColumn(
#     "dob_match_score_percentage",
#     col("dob_match_score") * 100
# )

In [0]:
display(df_with_dob_score)

ph_name,DOB_ph,bl_name,DOB_bl,name_match_score_percentage,DOB_ph_standardized,DOB_bl_standardized,dob_match_score
Khan Yusuf,1980-01-12,Yusuf Khan,19800113,100.0,1980-01-12,1980-01-13,50
Anil Rawat,1982-01-16,Aisha Rahmani,1990-03-22,65.47619047619047,1982-01-16,1990-03-22,0
Ayesha Rahman,1990/03/22,Aisha Rahmani,1990-03-22,90.57142857142857,1990-03-22,1990-03-22,100
Bhagyashree Khade,1983-07-22,Yusuf Khan,19800113,70.4469696969697,1983-07-22,1980-01-13,0
Bilal S.,1978/08/09,Bilal Sheikh,09/09/1987,87.5,1978-08-09,1987-09-09,0
Darwish Samira,27/05/1986,Samira Darwish,27-05-1986,100.0,1986-05-27,1986-05-27,100
Haddad Omar,05-11-1975,Omar Haddad,1975-11-05,100.0,1975-11-05,1975-11-05,100
Kareem Almasri,1982-07-15,Karim Al-Masri,1982-07-17,93.77777777777779,1982-07-15,1982-07-17,50
Mansour Leila,1992-09-18,Leila Mansour,1992/09/18,100.0,1992-09-18,1992-09-18,100
Noor Fatima,1988-06-30,Fatima Noor,1988-06-30,100.0,1988-06-30,1988-06-30,100


In [0]:
total_confidence_score = df_with_dob_score.withColumn("total_score", col("name_match_score_percentage") * 0.8 + col("dob_match_score") * 0.2).drop("DOB_ph", "DOB_bl")

In [0]:
display(total_confidence_score)

ph_name,bl_name,name_match_score_percentage,DOB_ph_standardized,DOB_bl_standardized,dob_match_score,total_score
Khan Yusuf,Yusuf Khan,100.0,1980-01-12,1980-01-13,50,90.0
Anil Rawat,Aisha Rahmani,65.47619047619047,1982-01-16,1990-03-22,0,52.38095238095238
Ayesha Rahman,Aisha Rahmani,90.57142857142857,1990-03-22,1990-03-22,100,92.45714285714286
Bhagyashree Khade,Yusuf Khan,70.4469696969697,1983-07-22,1980-01-13,0,56.357575757575766
Bilal S.,Bilal Sheikh,87.5,1978-08-09,1987-09-09,0,70.0
Darwish Samira,Samira Darwish,100.0,1986-05-27,1986-05-27,100,100.0
Haddad Omar,Omar Haddad,100.0,1975-11-05,1975-11-05,100,100.0
Kareem Almasri,Karim Al-Masri,93.77777777777779,1982-07-15,1982-07-17,50,85.02222222222223
Mansour Leila,Leila Mansour,100.0,1992-09-18,1992-09-18,100,100.0
Noor Fatima,Fatima Noor,100.0,1988-06-30,1988-06-30,100,100.0
